In [11]:
import os
import time
import requests
from typing import Any, Dict, List, Optional, Tuple

GEOCODE_URL = "https://maps.googleapis.com/maps/api/geocode/json"
PLACES_NEARBY_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

def get_api_key() -> str:
    key = os.getenv("GOOGLE_MAPS_API_KEY")
    if not key:
        raise RuntimeError("環境変数 GOOGLE_MAPS_API_KEY が未設定です。")
    return key

def geocode_station(station_name: str, api_key: str) -> Tuple[float, float]:
    q = station_name.strip()
    if not q:
        raise ValueError("駅名が空です。")

    params = {"address": q, "key": api_key, "language": "ja", "region": "jp"}
    r = requests.get(GEOCODE_URL, params=params, timeout=20)
    r.raise_for_status()
    data = r.json()

    status = data.get("status")
    if status != "OK" or not data.get("results"):
        raise RuntimeError(f"Geocoding失敗: status={status}, query={q}")

    loc = data["results"][0]["geometry"]["location"]
    return float(loc["lat"]), float(loc["lng"])

def fetch_restaurants_nearby(
    lat: float, lng: float, radius_m: int, api_key: str, max_results: int = 20
) -> List[Dict[str, Any]]:
    params = {
        "location": f"{lat},{lng}",
        "radius": radius_m,
        "type": "restaurant",
        "key": api_key,
        "language": "ja",
    }

    results: List[Dict[str, Any]] = []
    next_page_token: Optional[str] = None

    while True:
        if next_page_token:
            time.sleep(2)
            params["pagetoken"] = next_page_token

        r = requests.get(PLACES_NEARBY_URL, params=params, timeout=20)
        r.raise_for_status()
        data = r.json()

        status = data.get("status")
        if status not in ("OK", "ZERO_RESULTS"):
            raise RuntimeError(f"Places失敗: status={status}, error={data.get('error_message')}")

        results.extend(data.get("results", []))

        if len(results) >= max_results:
            return results[:max_results]

        next_page_token = data.get("next_page_token")
        if not next_page_token:
            return results

def print_restaurants(restaurants: List[Dict[str, Any]]) -> None:
    if not restaurants:
        print("半径500m以内にレストランが見つかりませんでした。")
        return

    for i, r in enumerate(restaurants, start=1):
        name = r.get("name", "（不明）")
        rating = r.get("rating", "（評価なし）")
        address = r.get("vicinity") or r.get("formatted_address") or "（住所不明）"

        print(f"{i}. レストラン名: {name}")
        print(f"   評価: {rating}")
        print(f"   住所: {address}")
        print("-" * 60)

def main():
    api_key = get_api_key()
    print("続いて、駅名を入力します")
    station = input("駅名検索欄: ").strip()

    lat, lng = geocode_station(station, api_key)
    restaurants = fetch_restaurants_nearby(lat, lng, radius_m=500, api_key=api_key, max_results=20)

    print("\n入力した駅から半径500メートル以内のレストラン情報を出力\n")
    print_restaurants(restaurants)

main()


続いて、駅名を入力します
駅名検索欄: 本町

入力した駅から半径500メートル以内のレストラン情報を出力

1. レストラン名: ニューハマヤ 瓦町店
   評価: 3.9
   住所: 大阪市中央区瓦町４丁目３−１０
------------------------------------------------------------
2. レストラン名: Grill & Kitchen かぼちゃの馬車
   評価: 4.1
   住所: 大阪市中央区本町３丁目４−１１ 丸寿ビル 地下1階
------------------------------------------------------------
3. レストラン名: 風泉
   評価: 3.7
   住所: 大阪市中央区本町４丁目５−１０ 長澤ビル ２F
------------------------------------------------------------
4. レストラン名: 千鳥屋宗家 大阪本店
   評価: 3.9
   住所: 大阪市中央区本町３丁目４−１２
------------------------------------------------------------
5. レストラン名: そばよし 本町店
   評価: 3.8
   住所: 大阪市中央区南本町３丁目６−１４ B2F
------------------------------------------------------------
6. レストラン名: 名代 宇奈とと 大阪本町店
   評価: 4
   住所: 大阪市中央区安土町３丁目４−２
------------------------------------------------------------
7. レストラン名: グリル 小松屋
   評価: 4
   住所: 大阪市中央区平野町４丁目６−１０
------------------------------------------------------------
8. レストラン名: うまいもんやつくし 本町店
   評価: 3.8
   住所: 大阪市中央区本町４丁目１−１３ 御堂ビルB1
-------------------------------------